In [1]:
# get libraries

import random

In [2]:
# global variables

# tuples (immutable lists)
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
dealers = ('Silas','Alex','Sean','Juliette','Remedios','Seth','Jeff','Hiroko','Robert','September',
           'Gennie','Lauran','Rosalba','Jaqueline','Christeen')
dealer_confused_responses = ('Sorry I do not understand...','I think I misheard you...','Come again...','Pardon me...')
dealer_lose_responses = ('Better luck next time!','Dealer wins!','Sorry!','Not this time my friend!')
dealer_win_responses = ('Well done!','Congrats!','Nicely played!','Kudos!')

# dictionaries
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 'Jack':10, 
          'Queen':10, 'King':10, 'Ace':11}

In [3]:
class Deck:
    '''The Deck class holds a list of card objects as an attribute and includes shuffle and deal methods
    to randomize and remove a card from the deck.'''
    
    def __init__(self):
        self.cards = [] # initialize list
        for rank in ranks:
            for suit in suits:
                # append each card from global variables
                self.cards.append(rank + ' of ' + suit)

    def shuffle(self):
        # in place
        random.shuffle(self.cards)
        
    def deal(self):
        # top (first in list) card removed
        return str(self.cards.pop(0)) 

In [4]:
class Hand:
    '''The Hand class creates instances of player and dealer card hands with card lists and values
    as attributes; methods for adding, clearing and showing cards and values; also adjust Ace value
    based on hand value.'''
    
    def __init__(self,name):
        self.name = name # will be dealer or player name
        self.cards = []  # initialize list of cards
        self.value = 0   # initialize value of cards
    
    def add_card(self,card):
        # add card name to list and value to total; note that if more than one card would use extend 
        # split from card string, handle Ace value, else lookup value from dictionary, accumulate
        if str(card).split()[0] == 'Ace' and self.value <= 10:
            self.value += 11
            self.cards.append(card)
        elif str(card).split()[0] == 'Ace' and self.value > 10:
            self.value += 1
            self.cards.append(card)
        else:
            self.value += values[ str(card).split()[0] ]
            self.cards.append(card)
        
    def clr_value(self):
        # reset card lists and value for a new hand
        self.cards.clear()
        self.value = 0
        
    def show_cards(self):
        # iterate through list in order to display
        for _ in self.cards:
            print(_)

In [5]:
class Chips:
    '''The Chips class holds an attribute of the player's chip value with methods to
    adds chips from cash prompt and adjust chips based on hand outcome.'''
    
    def __init__(self):
        self.value = 0   # initialize value of chips
        
    def chg_cash(self,cash):  
        self.value += int(cash)
        
    def chg_bet(self,wager,outcome):
        if outcome == 'win':
            self.value += wager
        elif outcome == 'push':
            pass # do nothing chip is returned
        else:
            self.value -= wager

In [6]:
# function to determine whether a game is to be played; prompt for incorrect responses

def game_on():
    player_response = ''
    while player_response.upper() not in ['Y','N','YES','NO']:
        player_response = input('\nWould you like to play again? ')
        if player_response.upper() in ['Y','YES']:
            return True
        elif player_response.upper() in ['N','NO']:
            return False
        else:
            print(random.choice(dealer_confused_responses))           
            continue          

In [7]:
# function to prompt for player to change cash to chips before starting the game 

def cash_change():
    while True:
        try:
            cash = float(input('\nHow much cash would you like to change for chips? '))
            if cash <=0:
                print(random.choice(dealer_confused_responses))      
                continue
            else:
                # int cast will drop decimals so as to only change whole dollars
                print(f'\nChanging ${int(cash)}!')
                return int(cash)
        except:
            print(random.choice(dealer_confused_responses))
            continue      

In [8]:
# funciton to prompt for player to make a wager on current action

def wager():
    while True:
        try:
            wager = input(f'\nHow much would you like to wager? You currently have {player_chips.value} chips. ')
            if float(wager) - int(float(wager)) > 0:
                print('Whole chips please.')
                continue
            elif int(wager) <= 0:
                print('You need to put something up to be in the game my friend.')
                continue
            elif int(wager) > player_chips.value:
                print('Sorry, you do not have enough chips.')
                continue
            else:
                return int(wager)
        except:
            # any character inputs will throw an int or float error in the try statements
            print(random.choice(dealer_confused_responses))      
            continue                

In [9]:
# function to prompt player to decide whether to hit or stand on current action

def plr_hit_stand():
    player_action = ''
    while player_action.upper() not in ['HIT','STAND']:
        player_action = input('\nWould you like to hit or stand? ')
        if player_action.upper() in ['HIT','STAND']:
            return player_action.upper()
        else:
            print(random.choice(dealer_confused_responses))
            continue

In [11]:
# main program

# dealer and player greetings
print('Welcome to Black Jack!')
dealer = Hand(name=random.choice(dealers))
player = Hand(name=input(f'\nI am {dealer.name}, your dealer today. What is your name? '))
print(f'\nHi {player.name.capitalize()}!')

# dealer prompts player to change cash for chips
player_chips = Chips()
player_chips.chg_cash(cash=cash_change())

# begin game play loop that ends when player has no more chips or no longer wants to play
while True:
    
    # end game if player would like to play but has no more chips
    if player_chips.value == 0:
        break
    
    # shuffle deck
    deck = Deck()
    deck.shuffle()
    
    # dealer obtains the player's wager
    player_wager = wager()
    print(f'\nPlayer bets {player_wager}!')
    
    # deal player 2 cards and add value to hand
    for _ in range(2):
        player.add_card( deck.deal() )
        
    # deal dealer 2 cards and add value to hand
    for _ in range(2):
        dealer.add_card( deck.deal() )
        
    # show 1 of dealer's cards 
    print('\nDealer shows ' + str(values[ str(dealer.cards[0]).split()[0] ] ) + '.')
    print(dealer.cards[0])
    
    # show both of player's cards
    print('\nPlayer has ' + str(player.value) + '.')
    player.show_cards()
    
    # if both player and dealer have 21 with just two cards dealt then push
    if player.value == 21 and player.value == dealer.value:
        print('\nDealer has ' + str(dealer.value) + '. Push!')
        dealer.show_cards()
        player_chips.chg_bet(wager=player_wager,outcome='push') # update player chip value 
    else:
        pass
    
    # if only player has 21 with two cards it is an automatic stand and dealer will hit until a push or bust
    if player.value == 21 and player.value > dealer.value:
        
        # show dealer cards before hitting
        print('\nDealer has ' + str(dealer.value) + '.')
        dealer.show_cards()
        
        # dealer hits until gets 21 or busts
        while dealer.value < 21:
            dealer.add_card( deck.deal() )
            print('\nDealer hits...')
            print(dealer.cards[-1])
            if dealer.value == 21:
                print('\nDealer has ' + str(dealer.value) + '. Push!')
                dealer.show_cards()
                player_chips.chg_bet(wager=player_wager,outcome='push') # update player chip value
                break
            elif dealer.value > 21:
                print('\nDealer has ' + str(dealer.value) + '. Dealer busts!')
                dealer.show_cards()
                player_chips.chg_bet(wager=player_wager,outcome='win') # update player chip value
                break
            else:
                continue
    else:
        pass
    
    # begin loop since more than 2 cards required to determine outcome
    while True:
        
        # player hits when prompted by dealer 
        if plr_hit_stand() == 'HIT':
            player.add_card( deck.deal() )
            print('\nPlayer hits...')
            print(player.cards[-1])
            
            # player hit results in a 21 push
            if player.value == 21 and player.value == dealer.value:
                print('\nPlayer has ' + str(player.value) + '.')
                player.show_cards()
                print('\nDealer has ' + str(dealer.value) + '. Push!')
                dealer.show_cards()
                player_chips.chg_bet(wager=player_wager,outcome='push') # update player chip value 
                break
            
            # player hit results in 21 and an automatic stand
            elif player.value == 21 and player.value > dealer.value:
        
                # show dealer cards before hitting
                print('\nDealer has ' + str(dealer.value) + '.')
                dealer.show_cards()
        
                # dealer hits until gets 21 or busts
                while dealer.value < 21:
                    dealer.add_card( deck.deal() )
                    print('\nDealer hits...')
                    print(dealer.cards[-1])
                    if dealer.value == 21:
                        print('\nDealer has ' + str(dealer.value) + '. Push!')
                        dealer.show_cards()
                        player_chips.chg_bet(wager=player_wager,outcome='push') # update player chip value
                        break
                    elif dealer.value > 21:
                        print('\nDealer has ' + str(dealer.value) + '. Dealer busts!')
                        dealer.show_cards()
                        player_chips.chg_bet(wager=player_wager,outcome='win') # update player chip value
                        break
                    else:
                        continue
            
            # player hit results in bust            
            elif player.value > 21:
                print('\nPlayer has ' + str(player.value) + '. Player busts!')
                player.show_cards()  
                player_chips.chg_bet(wager=player_wager,outcome='lose') # update player chip value 
                print('\nDealer has ' + str(dealer.value) + '.')
                dealer.show_cards()
                break
             
            # player hit value is insufficient to determine outcome
            else:
                print('\nPlayer has ' + str(player.value) + '.')
                player.show_cards()   
                print('\nDealer shows ' + str(values[ str(dealer.cards[0]).split()[0] ] ) + '.')
                print(dealer.cards[0])
                continue
                
        # player stands  
        else:
            print(f'\nPlayer stands at {str(player.value)}!')

            # players loses after after standing with existing cards
            if dealer.value > player.value:
                print('\nDealer has ' + str(dealer.value) + '. ' + random.choice(dealer_lose_responses))
                dealer.show_cards()
                player_chips.chg_bet(wager=player_wager,outcome='lose') # update player chip value
                break
            
            # player wins after standing with existing cards 
            elif dealer.value < player.value and dealer.value >= 17:
                print('\nDealer has ' + str(dealer.value) + '. ' + random.choice(dealer_win_responses))
                dealer.show_cards()
                player_chips.chg_bet(wager=player_wager,outcome='win') # update player chip value
                break
                
            # begin loop for dealer hitting until => 17 and break for win, lose or push outcome
            else:
                print('\nDealer has ' + str(dealer.value) + '.')
                dealer.show_cards()
                while dealer.value < 17:
                    dealer.add_card( deck.deal() )
                    print('\nDealer hits...')
                    print(dealer.cards[-1])
                    if dealer.value >= 17 and dealer.value <= 21 and dealer.value > player.value:
                        print('\nDealer has ' + str(dealer.value) + '. ' + random.choice(dealer_lose_responses))
                        dealer.show_cards()
                        player_chips.chg_bet(wager=player_wager,outcome='lose') # update player chip value
                        break
                    elif dealer.value >= 17 and dealer.value <= 21 and dealer.value == player.value:
                        print('\nDealer has ' + str(dealer.value) + '. Push!')
                        dealer.show_cards()
                        player_chips.chg_bet(wager=player_wager,outcome='push') # update player chip value
                        break
                    elif dealer.value > 21:
                        print('\nDealer has ' + str(dealer.value) + '. Dealer busts!')
                        dealer.show_cards()
                        player_chips.chg_bet(wager=player_wager,outcome='win') # update player chip value
                        break
                    elif dealer.value >= 17 and dealer.value <= 21 and dealer.value < player.value:
                        print('\nDealer has ' + str(dealer.value) + '. ' + random.choice(dealer_win_responses))
                        dealer.show_cards()
                        player_chips.chg_bet(wager=player_wager,outcome='win') # update player chip value
                        break
                    else:
                        continue # dealer keeps hitting
            break # break out of game loop after hand loop is complete

    # dealer prompts player to keep playing  
    if game_on() == True:
        player.clr_value() # reset player hand cards and value
        dealer.clr_value() # reset dealer hand cards and value
        continue # player wants to keep playing
    else:
        break # player does not want to keep playing

# dealer goodbye message for broke player        
if player_chips.value == 0:
    print("\nYou're broke. Thank you for playing.") 
# dealer goodbye message for player with chips remaining
else: 
    print(f'\nThank you for playing. You finished with ${str(player_chips.value)} in chips.')

Welcome to Black Jack!

I am Robert, your dealer today. What is your name? aaron

Hi Aaron!

How much cash would you like to change for chips? 10

Changing $10!

How much would you like to wager? You currently have 10 chips. 1

Player bets 1!

Dealer shows 8.
Eight of Diamonds

Player has 12.
Ten of Clubs
Two of Diamonds

Would you like to hit or stand? hit

Player hits...
Queen of Hearts

Player has 22. Player busts!
Ten of Clubs
Two of Diamonds
Queen of Hearts

Dealer has 18.
Eight of Diamonds
King of Diamonds

Would you like to play again? yes

How much would you like to wager? You currently have 9 chips. 9

Player bets 9!

Dealer shows 7.
Seven of Clubs

Player has 18.
Seven of Diamonds
Ace of Clubs

Would you like to hit or stand? stand

Player stands at 18!

Dealer has 17. Kudos!
Seven of Clubs
Ten of Spades

Would you like to play again? no

Thank you for playing. You finished with $18 in chips.
